In [1]:
# pip install gurobipy

In [26]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *

import numpy as np
import pandas as pd
from IPython.display import display

import math
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from tabulate import tabulate
import statistics as st
import random
import copy

import seaborn as sns

import time

In [27]:
def SEIR_simulation(
        
### (1) Input       
        
# (1.1) Population
    terminal_time,               #::Int64,
    susceptible_population,      #::Int64,
    exposed_population,          #::Int64,
    recovered_population,       #::Int64,
    infected_population,         #::Int64,
    cured_population,            #::Int64,
    deaded_population,           #::Int64,
        
# (1.2) contact rate & proportion
    contact_rate,                #::Vector,
    contact_rate_proportion,     #::Vector,   
        
# (1.3) sensitivity (SEIR parameters)
    infection_rate,              #::Vector,
    exposed_to_infected_rate,    #::Vector,
    recovery_rate_for_exposed,   #::Vector,
    cured_rate_for_infected,     #::Vector,
    death_rate,                  #::Vector,
    sensitivity_proportion,      #::Vector,
        
# (1.4) vaccination strategy
    vaccination_strategy,        #::Function,
    vaccine_efficacy,            #::Float64,
    vaccine_coverage_time,       #::Int64,
    total_time_for_greedy,
        
# (1.5) Output form
    output_presenting_function):  #::Function
    
    
### (2) Define parameters for all group divisions based on activity and sensitivity
    sensitivity_number = len(sensitivity_proportion)
    activity_number = len(contact_rate_proportion)
    N_0 = susceptible_population + exposed_population + recovered_population + infected_population + cured_population
    
    division_number = sensitivity_number * activity_number     # number of divisions
    c = np.zeros(division_number)          # contact rate
    p = np.zeros(division_number)          # proportion
    lambdas = np.zeros(division_number)          # lamda, Infection rate
    gammas = np.zeros(division_number)          # gamma, Exposed to infected rate
    sigma_Es = np.zeros(division_number)          # sigma_E, Recovery rate for exposed
    sigma_Is = np.zeros(division_number)          # sigma_I, Cured rate for infected
    deltas = np.zeros(division_number)          # delta, death rate (case fatality rate)

    for j in range(0,activity_number):
        for i in range(0,sensitivity_number):
            c[(j)*sensitivity_number+i] = contact_rate[j]
            p[(j)*sensitivity_number+i] = contact_rate_proportion[j]*sensitivity_proportion[i]
            lambdas[(j)*sensitivity_number+i] = infection_rate[i]
            gammas[(j)*sensitivity_number+i] = exposed_to_infected_rate[i]
            sigma_Es[(j)*sensitivity_number+i] = recovery_rate_for_exposed[i]
            sigma_Is[(j)*sensitivity_number+i] = cured_rate_for_infected[i]
            deltas[(j)*sensitivity_number+i] = death_rate[i]
        # end for i
    # end for j

# (2.1) Show all the parameters for each division (not be printed out)
    Sensitivity_Matrix = np.zeros((7, division_number))
    Sensitivity_Matrix[0] = p        ; Sensitivity_Matrix[1] = c      
    Sensitivity_Matrix[2] = lambdas  ; Sensitivity_Matrix[3] = gammas  
    Sensitivity_Matrix[4] = sigma_Es ; Sensitivity_Matrix[5] = sigma_Is
    Sensitivity_Matrix[6] = deltas 
    Sensitivity_Matrix_df = pd.DataFrame(Sensitivity_Matrix.T)
    colnames = ["Proportion","Contact rate","Infection rate","Exposed to infected","Recovery rate","Cured rate","Death rate"]
    Sensitivity_Matrix_df.columns = colnames
    
### (3) Prepare to record population in each division for all time
    S = np.zeros((terminal_time+1, division_number))
    dS = np.zeros((terminal_time+1, division_number))
    E = np.zeros((terminal_time+1, division_number))
    R = np.zeros((terminal_time+1, division_number)) 
    I = np.zeros((terminal_time+1, division_number))
    C = np.zeros((terminal_time+1, division_number))
    D = np.zeros((terminal_time+1, division_number))
    dD = np.zeros((terminal_time, division_number)) 
    N = np.zeros((terminal_time+1, division_number))
    V = np.zeros((terminal_time+1, division_number))         # V records vaccinated population of each period 
    Immunized = np.zeros((terminal_time+1, division_number)) # Immunized records the Cumulative vaccinated population 

### (4) Record the intial population 
    S[0,:] = p * susceptible_population    
    E[0,:] = p * exposed_population    
    R[0,:] = p * recovered_population
    I[0,:] = p * infected_population
    C[0,:] = p * cured_population
    D[0,:] = p * deaded_population
    N[0,:] = p * N_0

### (5) Epidemic Simulation
    
# (5.0) Decide maximum vaccine amount for each period
    vaccine_max = N_0/vaccine_coverage_time
    
    for t in range(1,terminal_time+1):  # 1 to terminal_time
    
# (5.1) Vaccination strategy (decide how many people vaccinated in each division)
        
        v = vaccination_strategy(t, 
                                 contact_rate, # this is the rawe input contact rate
                                 c, # this one is a renewed term for each division, others are raw input
                                 infection_rate,
                                 exposed_to_infected_rate,
                                 gammas, # this one is a renewed term for each division, others are raw input
                                 recovery_rate_for_exposed,
                                 cured_rate_for_infected,
                                 death_rate,
                                 vaccine_max, 
                                 S[t-1], E[t-1], R[t-1], I[t-1], N[t-1], Immunized[t-1], 
                                 total_time_for_greedy)
        
        V[t-1] = v                        # Record vaccinated people in period t
        Immunized[t] = sum(V)     # Record total vaccinated people till time t

# (5.1.1) Determine vaccinated proportion for each division among S, E, R states, in case we have 0/(0 + 0 + 0)
    
        vaccinated_proportion_in_S = np.zeros(division_number)
        vaccinated_proportion_in_E = np.zeros(division_number)
        vaccinated_proportion_in_R = np.zeros(division_number)
        
        for k in range(0,division_number): 
            if S[t-1,k] == 0:
                vaccinated_proportion_in_S[k] = 0
            else:
                vaccinated_proportion_in_S[k] = S[t-1,k]/(S[t-1,k]+E[t-1,k]+R[t-1,k])
            # end if
            if E[t-1,k] == 0:
                vaccinated_proportion_in_E[k] = 0
            else:
                vaccinated_proportion_in_E[k] = E[t-1,k]/(S[t-1,k]+E[t-1,k]+R[t-1,k])
            # end if
            if R[t-1,k] == 0:
                vaccinated_proportion_in_R[k] = 0
            else:
                vaccinated_proportion_in_R[k] = R[t-1,k]/(S[t-1,k]+E[t-1,k]+R[t-1,k])
            # end if
        # end for k
    
                    
# (5.2) determine delta S for each division k
  
        for k in range(0,division_number): 
            if S[t-1,k] == 0:   # no more S people, dS will be 0.
                dS[t-1,k] = 0
            else:
                source = sum(c*(E[t-1] - vaccine_efficacy * v * vaccinated_proportion_in_E)) # population contacted by exposed people
                dS[t-1,k] = min(lambdas[k] * (S[t-1,k] - vaccine_efficacy*v[k]*vaccinated_proportion_in_S[k])/(sum(N[t-1])-sum(I[t-1])) * source, S[t-1,k])
            # end if
        # end for k
        
# (5.3) SEIR change: S, E, R, I, C
    
        S[t] = S[t-1] - dS[t-1]                                    - vaccine_efficacy * v * vaccinated_proportion_in_S
        E[t] = E[t-1] + dS[t-1] - sigma_Es*E[t-1] - gammas*E[t-1] - vaccine_efficacy * v * vaccinated_proportion_in_E
        R[t] = R[t-1]           + sigma_Es*E[t-1]                  - vaccine_efficacy * v * vaccinated_proportion_in_R             
        I[t] = I[t-1]                              + gammas*E[t-1] - sigma_Is*I[t-1] - deltas*I[t-1]
        C[t] = C[t-1]                                              + sigma_Is*I[t-1]
        
        for k in range(0,division_number):        # in case the population decreases to negative
            S[t,k] = max(S[t,k], 0)   
            E[t,k] = max(E[t,k], 0)
            R[t,k] = max(R[t,k], 0)
        # end for k

# (5.4) Death occured in t-th period
    
        for k in range(0,division_number): 
            dD[t-1,k] = min(deltas[k]*I[t-1,k], N[t-1,k])
        # end for k
                                        
# (5.5) SEIR change: D, N  
        
        D[t] = D[t-1] + dD[t-1]
        N[t] = N[t-1] - dD[t-1]

    # end for t

### (6) Outcome

    return(output_presenting_function(S, E, R, I, C, D, V, Immunized, terminal_time))
    
# end def

In [28]:
def give_performance_table(susceptible_population,
                     exposed_population,
                     recovered_population,
                     infected_population,
                     cured_population,
                     deaded_population,
                     vaccinated_population,
                     immunized_population,
                     T_max):
    Performance_Matrix = np.zeros(7)
# highest infection population
    Performance_Matrix[0] = max(sum(infected_population.T))
# when highest I population appear
    Performance_Matrix[1] = np.argmax(sum(infected_population.T))
# highest infection proportion among all current alive people
    Performance_Matrix[2] = max(sum(infected_population.T)/(sum(susceptible_population.T)
                                                                +sum(exposed_population.T)
                                                                +sum(recovered_population.T)
                                                                +sum(infected_population.T)
                                                                +sum(cured_population.T)
                                                                +sum(vaccinated_population.T)))
# when does the highest I proportion appear
    Performance_Matrix[3] = np.argmax(sum(infected_population.T)/(sum(susceptible_population.T)
                                                                +sum(exposed_population.T)
                                                                +sum(recovered_population.T)
                                                                +sum(infected_population.T)
                                                                +sum(cured_population.T)
                                                                +sum(vaccinated_population.T)))
# Final death number
    Performance_Matrix[4] = sum(deaded_population.T)[-1]
# Final death rate
    Performance_Matrix[5] = (sum(deaded_population.T)[-1])/((sum(susceptible_population.T)
                                                            +sum(exposed_population.T)
                                                            +sum(recovered_population.T)
                                                            +sum(infected_population.T)
                                                            +sum(cured_population.T)
                                                            +sum(vaccinated_population.T))[0]) # divided by the initial population
# Total immunized population
    Performance_Matrix[6] = sum(immunized_population[T_max])
    return(Performance_Matrix)
# end def

In [29]:
def give_population_table(susceptible_population,
                    exposed_population,
                    recovered_population,
                    infected_population,
                    cured_population,
                    deaded_population,
                    vaccinated_population,
                    immunized_population,
                    T_max):
    Population_Matrix = np.zeros((7, T_max+1))
    Population_Matrix[0,:] = sum(susceptible_population.T)
    Population_Matrix[1,:] = sum(exposed_population.T)
    Population_Matrix[2,:] = sum(recovered_population.T)
    Population_Matrix[3,:] = sum(infected_population.T)
    Population_Matrix[4,:] = sum(cured_population.T)
    Population_Matrix[5,:] = sum(deaded_population.T)
    Population_Matrix[6,:] = sum(immunized_population.T)
    return(Population_Matrix)
# end def

3.2 Proposed vaccination - distribute vaccine based on division's priority, which is "contact rate * sensitivity".

In [30]:
def proposed_vaccination(t_th_period, 
                         unadjusted_contact,
                         contact, 
                        S_to_E_rate,
                        unadjusted_E_to_I_rate,
                         E_to_I_rate, 
                        E_to_R_rate,
                        I_to_C_rate,
                        I_to_D_rate,
                        max_vaccine_amount, 
                        S_t, E_t, R_t, I_t, N_t, V_t, 
                        T_max_for_greedy):
# initialization before allocating vaccine
    v = np.zeros(len(contact))
    
# defines vaccination priority for each division    
    dividsion_priority = np.zeros(len(contact))  
    for k in range(0,len(contact)):
        dividsion_priority[k] = contact[k] * E_to_I_rate[k]
    # end for k
    
# allocate vaccine based on priority
    for k in range(0,len(contact)):
        order_list = np.argsort(dividsion_priority).tolist() # which group has the lowest to highest priority
        order_list.reverse()                                 # reverse the order to highest to lowest priority
        chosen_group = order_list[k]
        if S_t[chosen_group] + E_t[chosen_group] > 0:
            v[chosen_group] = min((S_t[chosen_group] + E_t[chosen_group] + R_t[chosen_group]), max_vaccine_amount - sum(v)) 
        # end if
     # end for k
    
    return(v)
# end def

# V. SEIR Simulation Input

In [31]:
# Sensitivity parameters - how likely to get infected and develop symtoms or recover
Lambdas = [0.1, 0.1]     # Infection rate 0.02 ~ 0.6
Gammas = [1/5, 1/10]      # Exposed to infected rate 1/14 ~ 1/5
Sigma_Es = [1/14, 1/14]     # Recovery rate for exposed 1/14
Sigma_Is = [1/20, 1/10]     # Cured rate for infected 1/20 ~ 1/10
Deltas = [0.025, 0.025]   # Death rate (case fatality rate) 2.3% ~ 2.6%
p_i = [0.5, 0.5]       # proportion corresponding to sensitivity clan

# 5.2 Social activity parameters - based on number of contatcs
c_j = [25, 15]             # contact rates for different social activity level group
p_j = [0.5, 0.5]        # proportion corresponding to contact rates

#State population - how many people in each states defined by SEIR model
T = 50        # Terminal time in week
T_greedy = 4  # number of periods used for greedy
S_0 = 100000    # Initial number of susceptible
E_0 = 50        # Initial number of exposed
R_0 = 0         # Initial number of recovered from E
I_0 = 0         # Initial number of infected 
C_0 = 0         # Initial number of recovered from R
D_0 = 0         # Initial number of Death

# Vaccine Statistics Input
coverage_time = 100 # in weeks
v_epsilon = 0.9

#Chance constraint threshold
ALPHA = 0.05
dS_variation = np.array([0.99, 0.995, 1, 1.005, 1.01])
# dS_variation_prob = np.array([0.05, 0.2, 0.5, 0.2, 0.05])
dS_variation_prob = np.array([0.0, 0.0, 1, 0.0, 0.0])

In [32]:
simulation_test_population_c_and_s = SEIR_simulation(
        
### (1) Input       
        
# (1.1) Population
    terminal_time               = T,
    susceptible_population      = S_0,
    exposed_population          = E_0,
    recovered_population        = R_0,
    infected_population         = I_0,
    cured_population            = C_0,
    deaded_population           = D_0,
        
# (1.2) contact rate & proportion
    contact_rate                = c_j,
    contact_rate_proportion     = p_j,   
        
# (1.3) sensitivity (SEIR parameters)
    infection_rate              = Lambdas,
    exposed_to_infected_rate    = Gammas,
    recovery_rate_for_exposed   = Sigma_Es,
    cured_rate_for_infected     = Sigma_Is,
    death_rate                  = Deltas,
    sensitivity_proportion      = p_i,
    
# (1.4) vaccination strategy (Only thing need to change before running)
    vaccination_strategy        = proposed_vaccination,
    vaccine_efficacy            = v_epsilon,
    vaccine_coverage_time       = coverage_time,
    total_time_for_greedy       = T_greedy,

# (1.5) Output form (2nd part need to change before running, choose 1 of 2)
#     output_presenting_function  = plot_population_change
#     output_presenting_function  = give_population_df
    output_presenting_function  = give_population_table
#     output_presenting_function  = give_performance
#     output_presenting_function  = give_performance_table
    
    )
# simulation_test_population_c_and_s

In [33]:
# Infection rate 0.02 ~ 0.6
Lambda_set = [0.25, 0.25]

# Exposed to infected rate 1/14 ~ 1/5
Gamma_set = [[1/5, 1/10],
             [0.14, 1/10],
             [1/5, 0.07],
             [0.14, 0.07]]

# Recovery rate for exposed 1/14
Sigma_E_set = [1/14, 1/14]    
# Cured rate for infected 1/20 ~ 1/10
Sigma_I_set = [1/20, 1/10]    
# Death rate (case fatality rate) 2.3% ~ 2.6%
Delta_set = [0.026, 0.023] 

# proportion corresponding to sensitivity clan
# p_i_set = [[0.9, 0.1],[0.8, 0.2],[0.7, 0.3],[0.6, 0.4],[0.5, 0.5]] 
p_i_set = [[0.5, 0.5],[0.4, 0.6],[0.3, 0.7],[0.2, 0.8],[0.1, 0.9]]

# contact rates for different social activity level group
c_j_set = [[25, 15],
           [25, 10],
           [20, 10],
           [20, 5],
           [10, 5]]
# add rational behind of why we do such change
           
# proportion corresponding to contact rates
# p_j_set = [[0.9, 0.1],[0.8, 0.2],[0.7, 0.3],[0.6, 0.4],[0.5, 0.5]]    
p_j_set = [[0.5, 0.5],[0.4, 0.6],[0.3, 0.7],[0.2, 0.8],[0.1, 0.9]]

# Population
T = 100        # Terminal time in week
T_greedy = 1  # number of periods used for greedy


S_0 = 100000    # Initial number of susceptible
R_0 = 0         # Initial number of recovered from E
I_0 = 0         # Initial number of infected 
C_0 = 0         # Initial number of recovered from R
D_0 = 0         # Initial number of Death
# No N_0, it will be calculated in the simulation function

In [34]:
coverage_time = 100
v_epsilon = 1
E_proportion_set = [0.001, 0.002, 0.005, 0.01]

In [35]:
# table summarize all simulation result
# only run this for once
# num of row = num of E situation * num of sensitivity situation
# num of column = 9, 
# E, sensitivity_1, sensitivity_2, 
# proposed % win, proposed average of win, proposed average of lose
# s1c2 % win, s1c2 average of win, s1c2 average of lose
Summary_table_I = np.zeros((len(E_proportion_set) * len(Gamma_set),1 + len(Lambda_set) + 6))
Summary_table_D = np.zeros((len(E_proportion_set) * len(Gamma_set),1 + len(Lambda_set) + 6))

In [36]:
# record max I & total D for all cases regarding %E, s, p_s, c, p_c, strategy
previous_data_I_df = pd.DataFrame({})
previous_data_D_df = pd.DataFrame({})

In [37]:
# E_proportion_set = [0.001, 0.002, 0.005, 0.01]
E_index = 3 # 0, 1, 2, 3
E_0 = E_proportion_set[E_index] * S_0 
E_index

3

In [38]:
Gamma_index = 3 # 0, 1, 2, 3
Gammas = Gamma_set[Gamma_index] 
Gamma_index

3

In [39]:
# A matrix record all performance data for each simulation
# number of row = number of senstivity proportion * number of contact rate * number of performance evaluation index
# number of column = number of strategy * number of contact rate proportion

num_performance_index = 7 
# (0)Highest I,(1)When 1,(2)Highest I %,(3)When 2,(4)Total D,(5)Fatality rate,(6)Total vaccinated

num_strategy = 6 
# (0)proposed_vaccination,        (1)contact_rate_based_vaccination,
# (2)random_vaccination,          (3)sen_1st_con_2nd_vaccination,
# (4)con_1st_sen_2nd_vaccination, (5)optimal_vaccination

Result_matrix = np.zeros((len(p_i_set)*len(c_j_set)*num_performance_index, num_strategy*len(p_j_set)))

# record each winning and losing w.r.t highest I and total D
winning_situation_I_proposed = []
losing_situation_I_proposed = []
winning_situation_D_proposed = []
losing_situation_D_proposed = []

# record each winning and losing w.r.t highest I and total D
winning_situation_I_s1c2 = []
losing_situation_I_s1c2 = []
winning_situation_D_s1c2 = []
losing_situation_D_s1c2 = []


In [40]:
Max_I_matrix_proposed_no_greedy = np.zeros((len(p_i_set)*len(c_j_set), len(p_j_set)*2))
Death_matrix_proposed_no_greedy = np.zeros((len(p_i_set)*len(c_j_set), len(p_j_set)*2))

Max_I_matrix_s1c2_no_greedy = np.zeros((len(p_i_set)*len(c_j_set), len(p_j_set)*2))
Death_matrix_s1c2_no_greedy = np.zeros((len(p_i_set)*len(c_j_set), len(p_j_set)*2))

In [41]:
start_time = time.time()

for sen_prop_i in range(0, len(p_i_set)):
    for con_i in range(0, len(c_j_set)):
        for con_prop_i in range(0, len(p_j_set)):
            
        # 1 simulation for each strategy    
            # 1.1 C * S strategy
            simulation_population_c_and_s = SEIR_simulation(
                terminal_time               = T,
                susceptible_population      = S_0, exposed_population          = E_0,
                recovered_population        = R_0, infected_population         = I_0,
                cured_population            = C_0, deaded_population           = D_0,
        
                contact_rate                = c_j_set[con_i],
                contact_rate_proportion     = p_j_set[con_prop_i],   
        
                infection_rate              = Lambda_set,
                exposed_to_infected_rate    = Gammas,
                recovery_rate_for_exposed   = Sigma_E_set,
                cured_rate_for_infected     = Sigma_I_set,
                death_rate                  = Delta_set,
                sensitivity_proportion      = p_i_set[sen_prop_i],
    
                vaccination_strategy        = proposed_vaccination,
                vaccine_efficacy            = v_epsilon,
                vaccine_coverage_time       = coverage_time,
                total_time_for_greedy       = T_greedy,

                output_presenting_function  = give_performance_table
            )

            print("----------------------------------------------------------------------------------------------")
            print("exposed proportion = ", E_0/S_0)
            print("senstivity = ", Gammas)
            print("senstivity proportion = ", p_i_set[sen_prop_i])
            print("contact rate = ", c_j_set[con_i])
            print("contact proportion = ", p_j_set[con_prop_i])
            print("----------------------------------------------------------------------------------------------")
        
        # 2 Record the results into Result_matrix
        ## c * s strategy
            Result_matrix[sen_prop_i*len(c_j_set)*num_performance_index+con_i*num_performance_index:
                          sen_prop_i*len(c_j_set)*num_performance_index+con_i*num_performance_index+num_performance_index,
                         con_prop_i*num_strategy+0] = simulation_population_c_and_s
        ## greedy
#             Result_matrix[sen_prop_i*len(p_i_set)*num_performance_index+con_i*num_performance_index:
#                           sen_prop_i*len(p_i_set)*num_performance_index+con_i*num_performance_index+num_performance_index,
#                          con_prop_i*num_strategy+5] = simulation_population_greedy
            
    
    
########################################### if no greedy #################################################

    # 3 Record the Highest infection    
            lowest_max_infection_no_greedy = min(simulation_population_c_and_s[0])
            
    ## 3.1 Record the Highest infection for proposed strategy
            focus_1 = simulation_population_c_and_s[0] 
            
            if focus_1 == lowest_max_infection_no_greedy:
                Max_I_matrix_proposed_no_greedy[sen_prop_i*len(c_j_set)+con_i,con_prop_i*2] = 1
            else:
                Max_I_matrix_proposed_no_greedy[sen_prop_i*len(c_j_set)+con_i,con_prop_i*2+1] = (focus_1 - lowest_max_infection_no_greedy)/lowest_max_infection_no_greedy
                losing_situation_I_proposed.append((focus_1 - lowest_max_infection_no_greedy)/lowest_max_infection_no_greedy)


print("--- %s seconds ---" % (time.time() - start_time))

----------------------------------------------------------------------------------------------
exposed proportion =  0.01
senstivity =  [0.14, 0.07]
senstivity proportion =  [0.5, 0.5]
contact rate =  [25, 15]
contact proportion =  [0.5, 0.5]
----------------------------------------------------------------------------------------------


TypeError: 'numpy.float64' object is not iterable